In [ ]:
#!pip install catboost
#!pip install transformers

In [ ]:
import pandas as pd
import numpy as np

import pandas_profiling

import spacy
import pickle
import transformers as ppb

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

from tqdm import tqdm, notebook

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from catboost import Pool, CatBoostRegressor

In [ ]:
RAND = sum(ord(x) for x in 'NEVER SURRENDER')
tqdm.pandas()

HIDDEN_N = 600
ETA = 0.0001
BATCH = 100
EPOCH = 300

In [ ]:
df = pd.read_csv('/content/tocolab.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  244768 non-null  int64 
 1   FullDescription     244768 non-null  object
 2   LocationNormalized  244768 non-null  object
 3   Category            244768 non-null  object
 4   SalaryNormalized    244768 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 9.3+ MB


In [ ]:
df.columns = df.columns.str.replace('(.)([A-Z])', r'\1_\2', regex=True).str.lower()

In [ ]:
%%time
#http https www
df['full_description'] = df['full_description'].str.replace(r'((https?:\/\/)|w{3}).*?( |$)',' ', regex=True)
df['full_description'] = (df['full_description'].str.replace(r'[^A-Za-z\']',' ', regex=True)
                                                .str.lower()
                                                .str.strip()
)
df['full_description'] = df['full_description'].str.replace(r'\W{2,}',' ', regex=True)

CPU times: user 52.4 s, sys: 770 ms, total: 53.1 s
Wall time: 54.7 s


2000 последних элементов будет "тестовой", чисто для удобства проверять ручками. В этой части он не нужен.

In [ ]:
df = df[:-2000].reset_index(drop=True)

In [ ]:
df_train, df_valid = train_test_split(df, test_size=0.1, random_state=RAND)

In [ ]:
print(df_train.shape, df_valid.shape)

(218491, 5) (24277, 5)


Гридсерч кота по выбранным признакам, на полной без теста.

In [ ]:
X_train = df[['full_description','location_normalized','category']]
y_train = df['salary_normalized']
X_valid = df_valid[['full_description','location_normalized','category']]
y_valid = df_valid['salary_normalized']

In [ ]:
catboost_params = {
    'iterations': 5000,
    'eval_metric': 'RMSE',
    'verbose': 500,
    'random_state': RAND,
    'depth': 7, 
    'learning_rate': 0.02, 
}

In [ ]:
text_features = ['full_description']
cat_features = list(X_train.columns)
train_pool = Pool(
    X_train, 
    y_train, 
    cat_features=cat_features, 
    text_features=text_features
)
val_pool = Pool(
    X_valid, 
    y_valid, 
    cat_features=cat_features, 
    text_features=text_features
)

In [ ]:
text_features = ['full_description']
cat_features = list(X_train.columns)

In [48]:
model = CatBoostRegressor(**catboost_params)
model.fit(train_pool, eval_set=val_pool)

0:	learn: 17564.3858582	test: 17360.3761121	best: 17360.3761121 (0)	total: 1.09s	remaining: 1h 30m 55s
500:	learn: 12406.2289634	test: 12179.3478220	best: 12179.3478220 (500)	total: 7m 57s	remaining: 1h 11m 26s
1000:	learn: 11653.8725048	test: 11450.1561653	best: 11450.1561653 (1000)	total: 15m 47s	remaining: 1h 3m 7s
1500:	learn: 11218.0037291	test: 11031.3281849	best: 11031.3281849 (1500)	total: 23m 29s	remaining: 54m 45s
2000:	learn: 10904.0777193	test: 10731.1052868	best: 10731.1052868 (2000)	total: 31m 6s	remaining: 46m 37s
2500:	learn: 10652.6782106	test: 10491.3676933	best: 10491.3676933 (2500)	total: 38m 37s	remaining: 38m 35s
3000:	learn: 10438.3814130	test: 10284.9900073	best: 10284.9900073 (3000)	total: 46m 21s	remaining: 30m 52s
3500:	learn: 10250.4982641	test: 10106.8264267	best: 10106.8264267 (3500)	total: 54m 1s	remaining: 23m 7s
4000:	learn: 10081.3729808	test: 9945.2713079	best: 9945.2713079 (4000)	total: 1h 1m 39s	remaining: 15m 23s
4500:	learn: 9927.7296755	test: 979

In [49]:
model.score(val_pool)

0.6936952753663848

Из-за пересохранения модели cat-грид и выводы ячеек дальше немного поехало. Но параметров было побольше на GPU :) Оставила принцип того, как делалось.

In [50]:
model.save_model('cpu_catboost_model4.bin')

In [ ]:
X_train = df[['full_description','location_normalized','category']]
y_train = df['salary_normalized']

In [ ]:
train_pool = Pool(
    X_train, 
    y_train, 
    cat_features=cat_features, 
    text_features=text_features
)



In [ ]:
cat_model = CatBoostRegressor(**catboost_params)  #, task_type="GPU"
grid_search_result = cat_model.grid_search(grid, train_pool, cv=2)

0:	learn: -3.5100205	test: -3.5051956	best: -3.5051956 (0)	total: 73.5ms	remaining: 2m 26s
500:	learn: 0.2644826	test: 0.2733541	best: 0.2733541 (500)	total: 24.7s	remaining: 1m 13s
1000:	learn: 0.2675325	test: 0.2743896	best: 0.2743929 (996)	total: 51.1s	remaining: 51s
1500:	learn: 0.2690872	test: 0.2745481	best: 0.2745628 (1485)	total: 1m 15s	remaining: 25.1s
1999:	learn: 0.2703216	test: 0.2744605	best: 0.2745628 (1485)	total: 1m 42s	remaining: 0us

bestTest = 0.2745628119
bestIteration = 1485

0:	loss: 0.2745628	best: 0.2745628 (0)	total: 2m 16s	remaining: 15m 58s
0:	learn: -3.0029149	test: -2.9949758	best: -2.9949758 (0)	total: 70.1ms	remaining: 2m 20s
500:	learn: 0.2693532	test: 0.2745683	best: 0.2746051 (427)	total: 24.5s	remaining: 1m 13s
1000:	learn: 0.2725149	test: 0.2742176	best: 0.2746051 (427)	total: 49.1s	remaining: 49s
1500:	learn: 0.2747217	test: 0.2740276	best: 0.2746051 (427)	total: 1m 13s	remaining: 24.5s
1999:	learn: 0.2764189	test: 0.2737309	best: 0.2746051 (427)	to

In [ ]:
grid_search_result['params']

{'depth': 6, 'learning_rate': 0.1, 'l2_leaf_reg': 5.0}

In [ ]:
cat_model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,full_description,85.459559
1,category,8.765777
2,location_normalized,5.774665
